In [1]:
from hypersurface_tf import *
from generate_h import *
import sympy as sp
from pprint import pprint
from scipy.optimize import minimize
from sympy.utilities.iterables import flatten
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf

In [2]:
z0, z1, z2, z3, z4= sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4
HS = Hypersurface(Z, f, 10000)

In [3]:
k = 6
HS.set_k(k)

In [4]:
start_time = time.time()
factor = HS.integrate(lambda patch: patch.num_eta_tf('FS'), holomorphic=True, tensor=True)

In [5]:
factor = factor.numpy()

Normalization factor of eta

In [6]:
h_sym = get_sym_info(HS)

In [7]:
def train(HS, g, learning_rate, factor):
    with tf.GradientTape() as t:
        h = tf.matmul(g, g, adjoint_b=True)
        h = h/h[0][0]
        integration = HS.integrate(lambda patch: tf.abs(patch.num_eta_tf(h)/factor - 1), tensor=True)
    dg = t.gradient(integration, g)
    g.assign_sub(learning_rate * dg)

In [8]:
g0 = np.linalg.cholesky(param_to_matrix(initial_FS_param(HS, h_sym), h_sym))
g0 = np.array(g0, dtype=np.complex64)
with tf.device('/cpu:0'):
    g = tf.Variable(g0)

In [9]:
learning_rate = 20

In [10]:
HS_test = Hypersurface(Z, f, 10000)
HS_test.set_k(k)

In [11]:
test_old = 10

In [12]:
epochs = range(5000)
for epoch in epochs:
    train(HS, g, learning_rate, factor)

    if epoch % 20 == 0:
        h = tf.matmul(g, g, adjoint_b=True)
        h = h/h[0][0]
        integration = HS.integrate(lambda patch: tf.abs(patch.num_eta_tf(h)/factor - 1), tensor=True).numpy()
        test = HS_test.integrate(lambda patch: tf.abs(patch.num_eta_tf(h)/factor - 1), tensor=True).numpy()
        print('train:', integration)
        print('test:', test)
        if test > test_old:
            break
        test_old = test

InternalError: 2 root error(s) found.
  (0) Internal:  Dst tensor is not initialized.
	 [[{{node unknown_1/_6}}]]
	 [[PartitionedCall/PartitionedCall/MatMul_5/_8]]
  (1) Internal:  Dst tensor is not initialized.
	 [[{{node unknown_1/_6}}]]
0 successful operations.
0 derived errors ignored. [Op:__forward_num_eta_tf_7147]

Function call stack:
num_eta_tf -> num_eta_tf


In [ ]:
h_minimal = tf.matmul(g, g, adjoint_b=True)
h_minimal = h_t/h_t[0][0]
print(h_t[-1])

In [ ]:
sigma = HS.integrate(lambda patch: tf.abs(patch.num_eta_tf(h_minimal)/factor - 1), tensor=True).numpy()
delta_sigma = math.sqrt(HS.integrate(lambda patch: (tf.abs(patch.num_eta_tf(h_minimal)/factor - 1) - sigma)**2, 
                                     tensor=True).numpy() / HS.n_points)

In [ ]:
print(sigma)
print(delta_sigma)